<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course2/week1/Initialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization (초기화) #

2번째 코스 (Improving Deep Neural Networks)의 첫 번째 과제에 오신것을 환영합니다.

인공 신경망을 훈련할 때는 사전에 가중치 값을 초기화해야 합니다. 이 때 가중치 값을 잘 설정하면 학습에 큰 도움을 줄 수 있습니다.

만약 이 코스의 이전 강좌를 마쳤다면, 가중치를 초기화하는 방법을 이미 한번씩 구현해보았을 것입니다. 하지만 새로운 인공 신경망에 적합한 가중치를 초기화하기 위해선 어떻게 해야 할까요? 이 과제에서는, 가중치를 다르게 했을 때 학습 결과가 어떻게 달라지는지 확인해볼 것입니다.

적합한 초기화 방법은,
- 경사 하강법의 수렴속도를 높일 수 있습니다.
- 훈련 및 일반화 오류로 빠질 수 있는 확률을 낮춥니다.

시작하기 전, 아래 코드 블록을 실행시켜 패키지와 평면 데이터 세트를 불러오세요.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
from init_utils import sigmoid, relu, compute_loss, forward_propagation, backward_propagation
from init_utils import update_parameters, predict, load_dataset, plot_decision_boundary, predict_dec

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# load image dataset: blue/red dots in circles
train_X, train_Y, test_X, test_Y = load_dataset()

목표는, 빨간 점과 파란 점을 구분하는 분류 모델을 만드는 것입니다.

## 1. Neural Network Model ##

이번 과제에선 지난 코스에서 구현한 바 있는 3-layer 신경망을 사용할 것입니다. 이 신경망과, 아래에 있는 4개의 가중치 초기화 방법을 사용해서 학습을 시킨 후 결과를 비교해보겠습니다.

- 0으로 초기화 : `initialize` 인자를 `zeros`로 설정하세요.
- 랜덤 초기화 : `initialize` 인자를 `random`으로 설정하세요. 이렇게 초기화할 경우, 가중치 값이 큰 무작위 값으로 설정됩니다.
- `He` 초기화 : `initialize` 인자를 `he` 로 설정합니다. 이 초기화 방법은 *He et. al, 2015* 논문에서 언급된 바 있는 스케일 방법으로 조정된 무작위 값으로 설정됩니다.

**지시 사항**: 바로 아래 코드 블록은 빠르게 읽고, 다음으로 넘어가세요. 다음 파트에서 이 `model()` 함수를 직접 호출하면서 세 가지 초기화 방법을 구현할 예정입니다.

In [ ]:
  def model(X, Y, learning_rate = 0.01, num_iterations = 15000, print_cost = True, initialization = "he"):
    """
    Implements a three-layer neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (2, number of examples)
    Y -- true "label" vector (containing 0 for red dots; 1 for blue dots), of shape (1, number of examples)
    learning_rate -- learning rate for gradient descent 
    num_iterations -- number of iterations to run gradient descent
    print_cost -- if True, print the cost every 1000 iterations
    initialization -- flag to choose which initialization to use ("zeros","random" or "he")
    
    Returns:
    parameters -- parameters learnt by the model
    """
        
    grads = {}
    costs = [] # to keep track of the loss
    m = X.shape[1] # number of examples
    layers_dims = [X.shape[0], 10, 5, 1]
    
    # Initialize parameters dictionary.
    if initialization == "zeros":
        parameters = initialize_parameters_zeros(layers_dims)
    elif initialization == "random":
        parameters = initialize_parameters_random(layers_dims)
    elif initialization == "he":
        parameters = initialize_parameters_he(layers_dims)

    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID.
        a3, cache = forward_propagation(X, parameters)
        
        # Loss
        cost = compute_loss(a3, Y)

        # Backward propagation.
        grads = backward_propagation(X, Y, cache)
        
        # Update parameters.
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # Print the loss every 1000 iterations
        if print_cost and i % 1000 == 0:
            print("Cost after iteration {}: {}".format(i, cost))
            costs.append(cost)
            
    # plot the loss
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

## 2. Zero initalization ##

인공신경망에는 두 종류의 파라미터가 있습니다.
- 가중치 행렬 : $(W^{[1]}, W^{[2]}, W^{[3]}, ..., W^{[L-1]}, W^{[L]})$
- bias 벡터 : $(b^{[1]}, b^{[2]}, b^{[3]}, ..., b^{[L-1]}, b^{[L]})$

**연습 문제**: 모든 파라미터를 0으로 초기화하는 아래 함수를 완성하세요. 이후 이 초기화 방법은 대칭을 깨지(break symmetry) 못하기 때문에 잘 작동하지 않는 것을 확인할 수 있을 것입니다. 그것과는 별개로, 직접 구현해보면서 어떤 일이 일어나는지 확인해봅시다. `np.zeros((..,..))` 함수를 사용하세요.

In [ ]:
# GRADED FUNCTION: initialize_parameters_zeros 

def initialize_parameters_zeros(layers_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    """
    
    parameters = {}
    L = len(layers_dims)            # number of layers in the network
    
    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = None
        parameters['b' + str(l)] = None
        ### END CODE HERE ###
    return parameters

In [ ]:
parameters = initialize_parameters_zeros([3,2,1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected Output**:
<table> 
    <tr>
    <td>
    <b>W1</b>
    </td>
        <td>
    [[ 0.  0.  0.]
 [ 0.  0.  0.]]
    </td>
    </tr>
    <tr>
    <td>
    <b>b1</b>
    </td>
        <td>
    [[ 0.]
 [ 0.]]
    </td>
    </tr>
    <tr>
    <td>
    <b>W2</b>
    </td>
        <td>
    [[ 0.  0.]]
    </td>
    </tr>
    <tr>
    <td>
    <b>b2</b>
    </td>
        <td>
    [[ 0.]]
    </td>
    </tr>
</table> 

아래 코드 블록을 실행시켜, `zero initailzation` 으로 초기화한 파라미터를 15000번 반복하며 학습해 봅시다.

In [ ]:
parameters = model(train_X, train_Y, initialization = "zeros")
print ("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

성능이 매우 나쁜것을 볼 수 있습니다. 뿐만 아니라 cost가 거의 감소하지 않았습니다. 나쁘게 말하면, 이 알고리즘은 그냥 무작위로 때려맞추는 것과 크게 다른 성능을 가지지 않습니다. 왜일까요? 예측 과정과 decision boundary에 대해서 보다 자세히 살펴봅시다.

In [ ]:
print ("predictions_train = " + str(predictions_train))
print ("predictions_test = " + str(predictions_test))

In [ ]:
plt.title("Model with Zeros initialization")
axes = plt.gca()
axes.set_xlim([-1.5,1.5])
axes.set_ylim([-1.5,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

모델은 모든 샘플 데이터에 대해 0을 예측합니다.

일반적으로 모든 가중치를 0으로 초기화하면 네트워크가 대칭을 깨지(break symmetry) 못합니다. 즉, 각 층의 모든 뉴런이 동일한 값을 학습하고, 모든 층에 대해 $n^{[l]}=1$ 과 같은 값을 사용하여 신경망을 훈련하는 것과 같습니다. 이렇게 학습된 신경망은 로지스틱 회귀 하나로 훈련한 분류 모델과 성능면에서 차이가 없습니다.

<b>기억해야 할 것:</b>
- 가중치 행렬 $W^{[l]}$ 은, 대칭을 깨기(break symmetry) 위하여 무작위로 초기화되어야 합니다.
- 반면 bias $b^{[l]}$는 0으로 초기화해도 괜찮습니다. $W^{[l]}$가 무작위로 초기화되는 한, 대칭이 깨질 것이기 때문입니다.

## 3. Random initialization ##

대칭을 깨기 위해서, 가중치를 무작위로 초기화해보겠습니다. 이를 통해서 각 뉴런은 입력값에 따라 다른 함수로 학습될 수 있습니다. 이 연습 문제에서, 여러분은 가중치를 매우 큰 범위에서 무작위로 설정했을 때 무슨 일이 일어나는지 볼 수 있습니다.

**연습 문제**: 매우 큰 범위(10을 곱한 스케일) 내에서 가중치를 무작위로 초기화하는 아래 함수를 완성해보세요. 가중치에는 `np.random.randn(..) * 10`을, bias값에는 `np.zeros(..)`을 사용하세요. 이 과제 내에서는 고정된 시드를 사용하고 있기 때문에(`np.random.seed(..)` 의 인자가 고정되어 있음) 여러번 코드를 실행해도 늘 같은 랜덤값으로 초기화가 될 것입니다.

In [ ]:
# GRADED FUNCTION: initialize_parameters_random

def initialize_parameters_random(layers_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    """
    
    np.random.seed(3)               # This seed makes sure your "random" numbers will be the as ours
    parameters = {}
    L = len(layers_dims)            # integer representing the number of layers
    
    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = None
        parameters['b' + str(l)] = None
        ### END CODE HERE ###

    return parameters

In [ ]:
parameters = initialize_parameters_random([3, 2, 1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected Output**:

<table> 
    <tr>
    <td>
    <b>W1</b>
    </td>
        <td>
    [[ 17.88628473   4.36509851   0.96497468]
 [-18.63492703  -2.77388203  -3.54758979]]
    </td>
    </tr>
    <tr>
    <td>
    <b>b1</b>
    </td>
        <td>
    [[ 0.]
 [ 0.]]
    </td>
    </tr>
    <tr>
    <td>
    <b>W2</b>
    </td>
        <td>
    [[-0.82741481 -6.27000677]]
    </td>
    </tr>
    <tr>
    <td>
    <b>b2</b>
    </td>
        <td>
    [[ 0.]]
    </td>
    </tr>

</table> 

아래 코드 블록을 실행시켜, `random initailzation` 으로 초기화한 파라미터를 15000번 반복하며 학습해 봅시다.

In [ ]:
parameters = model(train_X, train_Y, initialization = "random")
print ("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

0번째 반복에서 cost로 `inf`가 표시되면, 이는 굉장히 큰 숫자가 반올림되었기 때문입니다. 보다 정교한 숫자 구현이 이 문제를 해결할 수 있습니다. 그러나 이 문제는 크게 걱정할 필요가 없습니다. 

어쨌든 대칭이 끊어진 것 처럼 보이며, 이전보다 더 나은 결과를 출력합니다. 모델이 더이상 모두 0을 출력하지는 않습니다.



In [ ]:
print (predictions_train)
print (predictions_test)

In [ ]:
plt.title("Model with large random initialization")
axes = plt.gca()
axes.set_xlim([-1.5,1.5])
axes.set_ylim([-1.5,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

**Observations**:
- cost 값이 매우 큰 값에서 시작됩니다. 이는 범위를 매우 크게 잡고 가중치를 무작위로 초기화했기 때문에, 최종 활성화 `sigmoid` 함수가 0 혹은 1에 매우 가까운 값을 출력하기 때문입니다. 만약 샘플이 잘못되면, 그 샘플에 대해 매우 높은 cost가 발생합니다. 실제로 $\log(a^{[3]}) = \log(0)$이면 손실은 무한대가 됩니다.
- 만약 초기화를 잘못하게 되면, v`anishing/exploding gradient` 문제가 발생하여 알고리즘을 최적화하는데 오래 걸릴 수 있습니다.
- 이 신경망을 더 오래 훈련시키면 더 좋은 결과를 얻을 수는 있겠지만, 가중치값을 큰 범위로 랜덤 초기화시켰기 때문에 최적화 속도를 느리게 만들 것입니다.

<br>

**요약하자면**:
- 매우 큰 범위에서 가중치를 무작위로 초기화하는 것은 잘 작동하지 않습니다.
- 작은 범위에서 가중치를 무작위로 초기화 하는 것은 효과가 있을까요? 중요한 부분은 얼마나 이 무작위 범위를 작게 설정해야 효과가 있느냐는 물음일 것입니다. 다음 파트에서 알아보겠습니다.

## 4. He initialization ##

마지막으로 **"He initialization""** 방법으로 파라미터를 초기화해보겠습니다. 이 이름은 *He et al., 2015* 의 첫 번째 저자의 이름을 따서 명명되었습니다. 만약 "Xavier initialization" 에 대해서 들어본 적이 있다면, Xavier 초기화가 가중치 행렬 $W^{[l]}$에 대해 `sqrt(1./layers_dims[l-1]` 의 scaling factor를 사용한다는 점을 제외하면 거의 유사합니다.

**연습 문제**: `He initialization` 방법으로 가중치를 초기화하는 아래 함수를 완성해보세요.

**힌트**: 이 함수는 이전 `initialize_parameters_random()` 함수와 유사합니다. 유일한 차이점은, `np.random.randn()` 을 한 가중치 값에 10을 곱하는것이 아니라 $\sqrt{\frac{2}{\text{dimension of the previous layer}}}$ 를 곱한다는 것입니다. 이 값은 ReLU 활성화 함수가 있는 층에 대해서 He initialization이 권장하는 값입니다.

In [ ]:
# GRADED FUNCTION: initialize_parameters_he

def initialize_parameters_he(layers_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the size of each layer.
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    W1 -- weight matrix of shape (layers_dims[1], layers_dims[0])
                    b1 -- bias vector of shape (layers_dims[1], 1)
                    ...
                    WL -- weight matrix of shape (layers_dims[L], layers_dims[L-1])
                    bL -- bias vector of shape (layers_dims[L], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layers_dims) - 1 # integer representing the number of layers
     
    for l in range(1, L + 1):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = None
        parameters['b' + str(l)] = None
        ### END CODE HERE ###
        
    return parameters

In [ ]:
parameters = initialize_parameters_he([2, 4, 1])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected Output**:
<table> 
    <tr>
    <td>
    <b>W1</b>
    </td>
        <td>
    [[ 1.78862847  0.43650985]
 [ 0.09649747 -1.8634927 ]
 [-0.2773882  -0.35475898]
 [-0.08274148 -0.62700068]]
    </td>
    </tr>
    <tr>
    <td>
    <b>b1</b>
    </td>
        <td>
    [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
    </td>
    </tr>
    <tr>
    <td>
    <b>W2</b>
    </td>
        <td>
    [[-0.03098412 -0.33744411 -0.92904268  0.62552248]]
    </td>
    </tr>
    <tr>
    <td>
    <b>b2</b>
    </td>
        <td>
    [[ 0.]]
    </td>
    </tr>
</table> 

아래 코드 블록을 실행시켜, `He initailzation` 으로 초기화한 파라미터를 15000번 반복하며 학습해 봅시다.

In [ ]:
parameters = model(train_X, train_Y, initialization = "he")
print ("On the train set:")
predictions_train = predict(train_X, train_Y, parameters)
print ("On the test set:")
predictions_test = predict(test_X, test_Y, parameters)

In [ ]:
plt.title("Model with He initialization")
axes = plt.gca()
axes.set_xlim([-1.5,1.5])
axes.set_ylim([-1.5,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

** Observations**:
- `He initialization`을 사용하는 모델은 반복 횟수를 적게 학습해도 파란 점과 빨간 점을 잘 분리합니다.

## 5. Conclusions ##

세 가지 가중치 초기화 방법이 어떻게 다른지 살펴보았습니다. 아래는 다른 조건(하이퍼파라미터 등) 이 동일하고 같은 횟수만큼 반복하여 학습 할 때, 세 초기화 방법을 비교한 표입니다.:

<table> 
    <tr>
        <td>
        <b>Model</b>
        </td>
        <td>
        <b>Train accuracy</b>
        </td>
        <td>
        <b>Problem/Comment</b>
        </td>
    </tr>
        <td>
        3-layer NN with zeros initialization
        </td>
        <td>
        50%
        </td>
        <td>
        fails to break symmetry
        </td>
    <tr>
        <td>
        3-layer NN with large random initialization
        </td>
        <td>
        83%
        </td>
        <td>
        too large weights 
        </td>
    </tr>
    <tr>
        <td>
        3-layer NN with He initialization
        </td>
        <td>
        99%
        </td>
        <td>
        recommended method
        </td>
    </tr>
</table> 

**이 과제를 통해 기억해야 할 것**:
- 초기화 방법을 달리 하면 모델의 학습 결과도 달라집니다.
- 무작위로 가중치를 초기화하면, 대칭을 깰 수 있어(break symmetry) 각 은닉층이 다른 것을 학습 할 수 있도록 합니다.
- 가중치를 너무 큰 값으로 초기화하면 안됩니다.
- `He initialization` 은 ReLU 활성화 함수에서 잘 작동하는 초기화 방법입니다.